In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# ===== ENV =====
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

llm = ChatGroq(model="llama-3.3-70b-versatile")

result = llm.invoke("What is 2 + 2?")
result


e:\anaconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='2 + 2 = 4.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 43, 'total_tokens': 52, 'completion_time': 0.012032902, 'completion_tokens_details': None, 'prompt_time': 0.001168913, 'prompt_tokens_details': None, 'queue_time': 0.057121647, 'total_time': 0.013201815}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfd5c-f6b5-72c2-a1d1-7c6842e36f1b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 43, 'output_tokens': 9, 'total_tokens': 52})

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list= [item for sublist in docs for item in sublist]

text_splitter   = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

doc_splits = text_splitter.split_documents(docs_list)

vectorstore     = FAISS.from_documents(doc_splits, embeddings)

retriever = vectorstore.as_retriever()



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
  """Binary score for relevance check on retrieved documents."""

  binary_score: str = Field(
    description="Documents are relevant to the question, 'yes' or 'no'."
  )

llm = ChatGroq(model="llama-3.3-70b-versatile")
structured_llm_grade = llm.with_structured_output(GradeDocuments)

system = """You are an expert assistant that helps users find relevant information from a set of documents to answer their questions. Your task is to evaluate the relevance of the provided documents in relation to the user's question."""
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human","Retrived documents: {documents}\n\nUser question: {question}")
  ])

retriever_grader= grade_prompt | structured_llm_grade
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retriever_grader.invoke({"question": question, "documents": doc_txt}))

binary_score='yes'


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# 1. Khởi tạo LLM
llm = ChatGroq(model="llama-3.3-70b-versatile")

template_text = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Question: {question} 
Context: {context} 
Answer:"""

prompt = ChatPromptTemplate.from_template(template_text)

# 3. Hàm xử lý format
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = prompt | llm | StrOutputParser()



formatted_context = format_docs(docs) 
generation = rag_chain.invoke({
    "context": formatted_context, 
    "question": question
})

print(generation)

Agent memory refers to the component of a LLM-powered autonomous agent system that enables agents to behave conditioned on past experience. The memory mechanism is a key component of the agent system, complementing the LLM brain. The context mentions "Types of Memory" and "Maximum Inner Product Search (MIPS)" as related concepts, but does not provide further details on how agent memory works.


In [10]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

system = """You are an expert assistant that helps users find relevant information from a set of documents to answer their questions. Your task is to evaluate the relevance of the provided documents in relation to the user's question. If the documents are not relevant, respond with 'no'. If they are relevant, respond with 'yes'."""

re_writer_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human","Here is the initial question: \n\n {question} \n Fromulate an improved question."),
  ])
question_rewriter = re_writer_prompt | llm | StrOutputParser()

question_rewriter.invoke({"question": question})

'Here\'s a more specific and improved question: \n\n"What are the current advancements and techniques in artificial intelligence for improving agent memory in multi-agent systems?"'